In [2]:
import numpy as np
import re

from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

# Задані тексти
texts_true = [
    "Обожнюю їсти! Мені подобається пробувати нові страви та кухні.",
    "Вчора я приготував лазанью, і вона вийшла дуже смачною.",
    "Обожнюю піцу і можу їсти її щодня."
]

texts_false = [
    "Не люблю готувати, тому харчуюся в основному в ресторанах.",
    "Я вегетаріанець, і мені складно знайти їжу в деяких країнах.",
    "У мене алергія на багато продуктів, тому мені складно їсти в ресторанах."
]

# Об'єднання текстів
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

print(list(tokenizer.word_index.items()))

X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indices = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indices]
Y = Y[indices]

model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length=max_text_len))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(2, activation='sigmoid'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=RMSprop(lr=0.00005))

history = model.fit(X, Y, batch_size=64, epochs=50)

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)


3 3 6
[('обожнюю', 2), ('їсти', 3), ('мені', 3), ('подобається', 1), ('пробувати', 1), ('нові', 1), ('страви', 1), ('та', 1), ('кухні', 1), ('вчора', 1)]
Обожнюю їсти! Мені подобається пробувати нові страви та кухні.
[[ 0  5  2  3 10 11 12 13 14 15]
 [ 0 16  6 17 18  4 19 20 21 22]
 [ 0  0  0  5 23  4 24  2 25 26]
 [ 0 27 28 29  7 30  1 31  1  8]
 [ 6 32  4  3  9 33 34  1 35 36]
 [39 40 41 42  7  3  9  2  1  8]]
[('в', 1), ('їсти', 2), ('мені', 3), ('і', 4), ('обожнюю', 5), ('я', 6), ('тому', 7), ('ресторанах', 8), ('складно', 9), ('подобається', 10), ('пробувати', 11), ('нові', 12), ('страви', 13), ('та', 14), ('кухні', 15), ('вчора', 16), ('приготував', 17), ('лазанью', 18), ('вона', 19), ('вийшла', 20), ('дуже', 21), ('смачною', 22), ('піцу', 23), ('можу', 24), ('її', 25), ('щодня', 26), ('не', 27), ('люблю', 28), ('готувати', 29), ('харчуюся', 30), ('основному', 31), ('вегетаріанець', 32), ('знайти', 33), ('їжу', 34), ('деяких', 35), ('країнах', 36), ('у', 37), ('мене', 38), ('алер

Epoch 1/50
1/1 [==============================] - 4s 4s/step - loss: 0.6924 - accuracy: 0.6667
Epoch 2/50
1/1 [==============================] - 0s 17ms/step - loss: 0.6857 - accuracy: 0.6667
Epoch 3/50
1/1 [==============================] - 0s 18ms/step - loss: 0.6770 - accuracy: 0.8333
Epoch 4/50
1/1 [==============================] - 0s 15ms/step - loss: 0.6611 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 15ms/step - loss: 0.6280 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 17ms/step - loss: 0.5535 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 17ms/step - loss: 0.3984 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 17ms/step - loss: 0.1822 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 18ms/step - loss: 0.0655 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 22ms/step - loss: 0.0347 - accuracy: 1.0000
Epoch 11/50
1/1 [==============

In [3]:
t = "Я люблю їсти смачну їжу.".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print(sequence_to_text(data[0]))

res = model.predict(data_pad)
print(res, np.argmax(res), sep='\n')

['я', 'люблю', 'їсти', 'їжу']
1/1 [==============================] - 1s 977ms/step
[[0.97529864 0.0437435 ]]
0


In [4]:
# Визначення результату
if np.argmax(res) == 0:
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")

Результат. Текст позитивний.


In [5]:
t1 = "Я не їм горіхи, бо в мене алергія на них.".lower()
data1 = tokenizer.texts_to_sequences([t1])
data1_pad = pad_sequences(data1, maxlen=max_text_len)
print( sequence_to_text(data1[0]) )

res1 = model.predict(data1_pad)


['я', 'не', 'в', 'мене', 'алергія', 'на']
1/1 [==============================] - 0s 19ms/step


In [6]:
# Визначення результату
if np.argmax(res1) == 0:
    print("Результат. Текст негативний.")
else:
    print("Результат. Текст позитивний.")

Результат. Текст негативний.


Висновок: на лабораторній роботі я навчився створювати RNN для семантичного аналізу з використанням моделі LSTM.